In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 모델 로드

In [2]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

In [3]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

#### 요약문 생성

In [4]:
from torch.utils.data import DataLoader
import pandas as pd

In [5]:
# 테스트 데이터셋 로드
df = pd.read_csv("/content/drive/MyDrive/응용자연어처리/project/data/medium_test.csv")

In [6]:
# 배치 요약문 생성 함수
def generate_batch_summaries(model, tokenizer, texts, device, batch_size=16):
    dataloader = DataLoader(texts, batch_size=batch_size)
    summaries = []
    for batch in dataloader:
        # 텍스트를 배치로 토큰화
        inputs = tokenizer(
            list(batch),
            return_tensors="pt",
            max_length=1024,
            truncation=True,
            padding="max_length"
        ).to(device)

        # 모델로 요약 생성
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128,
            num_beams=4,
            early_stopping=True
        )

        # 요약문 디코딩
        batch_summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        summaries.extend(batch_summaries)
    return summaries

# GPU/CPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# 배치 처리 및 요약문 생성
batch_size = 16
df["generated_summary"] = generate_batch_summaries(
    model, tokenizer, df["contents"].tolist(), device, batch_size
)

In [7]:
df.head()

,contents,summary,char_length,length,generated_summary
0,황인옥 세계여행 선언한 주부 이야기 대한민국연극제 대구 대표 참가 최우수연기상 ‘툇...,지난 31일에 열린 시상식에서 극단 온누리가 12년 만의 출전으로 제36회 대구연극...,1017,medium,지난달 22~31일까지 열린 제36회 대구연극제에는 5개 극단이 참가했다.
1,임재업 신ㆎ 구 이사장간 '고소전' 치달아 (동양일보 임재업 기자) 청주첨단산업용품...,청주 첨단산업용품 상사 협동조합이 청주 테크노폴리스 지역에 조성 예정인 산업용품단지...,1069,medium,청주첨단산업용품상사협동조합(이하 조합)이 청주테크노폴리스지역에 조성 예정인 산업용품...
2,도-경기관광公 ‘경기그랜드투어-해설이 있는 여행’ 출시 ▲ 경기관광공사가 명사와 함...,"경기도와 경기관광공사는 남한산성 성곽길을 걸으며 역사 이야기를 듣고, 화담숲에서 나...",1076,medium,경기도와 경기관광공사는 '경기그랜드투어-해설이 있는 여행'을 출시했다고 16일 밝혔다.
3,[서울경제] 최근 경기 악화로 지난해 경북 김천·구미 지역에서 노동청에 임금 체불 ...,"구미 김천지역 체불임금이 5년전보다 2개 가까이 증가한 것으로 나타나, 구미고용노동...",1005,medium,최근 경기 경기 악화로 지난해 경북 김천·구미 지역에서 노동청에 임금 체불 신고를 ...
4,19일 여당인 더불어민주당은 조국 법무부 장관 후보자를 향해 야당이 제기한 의혹에 ...,"19일 더불어민주당은 조국 법무부 장관 후보자를 향해 야당이 제기한 의혹에 대해 ""...",1439,medium,"더불어민주당은 조국 법무부 장관 후보자를 향해 야당이 제기한 의혹에 대해 ""무분별한..."


In [8]:
summaries = df["summary"]
generated_summaries = df["generated_summary"]

#### 성능평가 - medium

ROUGE: https://github.com/HeegyuKim/korouge

In [9]:
!pip install korouge_score

In [10]:
import numpy as np
from korouge_score import rouge_scorer

In [11]:
# 한국어 ROUGE
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL", "rougeLsum"])

In [12]:
# ROUGE 점수 계산
rouge_scores = []
for reference, generated in zip(df["summary"], df["generated_summary"]):
    scores = scorer.score(reference, generated)
    rouge_scores.append(scores)

# 결과 확인
for i, scores in enumerate(rouge_scores[:5]):
    print(f"Sample {i + 1}:")
    print(scores)
    print()

Sample 1:
{'rouge1': Score(precision=0.25, recall=0.1, fmeasure=0.14285714285714288), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.25, recall=0.1, fmeasure=0.14285714285714288), 'rougeLsum': Score(precision=0.25, recall=0.1, fmeasure=0.14285714285714288)}

Sample 2:
{'rouge1': Score(precision=0.3333333333333333, recall=0.42857142857142855, fmeasure=0.375), 'rouge2': Score(precision=0.19230769230769232, recall=0.25, fmeasure=0.2173913043478261), 'rougeL': Score(precision=0.25925925925925924, recall=0.3333333333333333, fmeasure=0.2916666666666667), 'rougeLsum': Score(precision=0.25925925925925924, recall=0.3333333333333333, fmeasure=0.2916666666666667)}

Sample 3:
{'rouge1': Score(precision=0.625, recall=0.16666666666666666, fmeasure=0.2631578947368421), 'rouge2': Score(precision=0.42857142857142855, recall=0.10344827586206896, fmeasure=0.16666666666666663), 'rougeL': Score(precision=0.625, recall=0.16666666666666666, fmeasure=0.2631578947368421),

In [13]:
# ROUGE 점수 평균 계산 (fmeasure 평균)
metrics = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
average_scores = {metric: 0 for metric in metrics}

# 총 샘플 수
n = len(rouge_scores)

# 각 메트릭의 fmeasure 합산
for score in rouge_scores:
    for metric in metrics:
        average_scores[metric] += getattr(score[metric], "fmeasure", 0)

# 각 메트릭의 평균 계산
for metric in metrics:
    average_scores[metric] /= n

print(average_scores)

{'rouge1': 0.27590679267846213, 'rouge2': 0.16645117218140115, 'rougeL': 0.2590588878924288, 'rougeLsum': 0.25917980033559107}


BERT-Score: https://github.com/Tiiiger/bert_score

In [14]:
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

In [15]:
# BERT-Score
bert_tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")
bert_model = BertModel.from_pretrained("beomi/kcbert-base").eval()

def get_bert_embeddings(sentence, tokenizer, model):
    # 문장을 토큰화하고 BERT 입력 형식으로 변환
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)

    # BERT 모델을 통해 문장의 임베딩 얻기
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = torch.mean(outputs.last_hidden_state, dim=1).squeeze(0)

    return embeddings.numpy()

def calculate_bert_score(sentence1, sentence2, tokenizer, model):
    # 각 문장의 BERT 임베딩 벡터 얻기
    embedding1 = get_bert_embeddings(sentence1, tokenizer, model)
    embedding2 = get_bert_embeddings(sentence2, tokenizer, model)

    # 코사인 유사도 계산 (1 - cosine distance)
    score = 1 - cosine(embedding1, embedding2)

    return score

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [16]:
# BERT-Score 점수 계산
df["bert_score"] = df.apply(lambda row: calculate_bert_score(row["summary"], row["generated_summary"], bert_tokenizer, bert_model), axis=1)
print(df.head())

                                            contents  \
0  황인옥 세계여행 선언한 주부 이야기 대한민국연극제 대구 대표 참가 최우수연기상 ‘툇...   
1  임재업 신ㆎ 구 이사장간 '고소전' 치달아 (동양일보 임재업 기자) 청주첨단산업용품...   
2  도-경기관광公 ‘경기그랜드투어-해설이 있는 여행’ 출시 ▲ 경기관광공사가 명사와 함...   
3  [서울경제] 최근 경기 악화로 지난해 경북 김천·구미 지역에서 노동청에 임금 체불 ...   
4  19일 여당인 더불어민주당은 조국 법무부 장관 후보자를 향해 야당이 제기한 의혹에 ...   

                                             summary  char_length  length  \
0  지난 31일에 열린 시상식에서 극단 온누리가 12년 만의 출전으로 제36회 대구연극...         1017  medium   
1  청주 첨단산업용품 상사 협동조합이 청주 테크노폴리스 지역에 조성 예정인 산업용품단지...         1069  medium   
2  경기도와 경기관광공사는 남한산성 성곽길을 걸으며 역사 이야기를 듣고, 화담숲에서 나...         1076  medium   
3  구미 김천지역 체불임금이 5년전보다 2개 가까이 증가한 것으로 나타나, 구미고용노동...         1005  medium   
4  19일 더불어민주당은 조국 법무부 장관 후보자를 향해 야당이 제기한 의혹에 대해 "...         1439  medium   

                                   generated_summary  bert_score  
0          지난달 22~31일까지 열린 제36회 대구연극제에는 5개 극단이 참가했다.    0.831142  
1  청주첨단산업용품상사협동조합(이하 조합)이 청주테크노폴리스지역에 조성 예정인 산업용품...    0.925309  

In [17]:
# BERT-Score 평균
df["bert_score"].mean()

0.8422200924934039